# Description

Notebook for loading simulation results for Figure 2.

# 1. Imports

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import os
import pandas as pd
import pingouin as pg
import pickle
import seaborn as sns
import sys

from tqdm import tqdm

# user imports
sys.path.append("../../")

from utils.pwr import *

# 2. Load simulated results data

## Simulation constants

In [2]:
n_trials = 500
fuzzy_gaps = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alpha = 0.05

low_cutoff = 0.25
hi_cutoff = 0.75

seeds = range(0, 401, 100)

In [3]:
RESULTS_DIR = "../../results/aaai/"

## Baseline discovery

In [41]:
baseline_result_dict = {}

for fuzzy_gap in tqdm(fuzzy_gaps):
    error_dict = {
        'x': {
            'fp': 0,
            'lower_fn': 0,
            'upper_fn': 0
        },

        'covar': {
            'fp': 0
        },
    }
    pvals = {}
    for seed in seeds:
        with open(f"../../experiments/aaai/baseline_discovery/seed{seed}/blended_rdd_fixed_bw_{fuzzy_gap}.pkl", "rb") as f:
            results = pickle.load(f)
            for result in results:
                x_thresholds = [np.round(c, decimals=2) for c in result['x'].keys()]
                if low_cutoff not in x_thresholds:
                    error_dict['x']['lower_fn'] += 1
                else: 
                    x_thresholds.remove(0.25)

                if hi_cutoff not in x_thresholds:
                    error_dict['x']['upper_fn'] += 1
                else:
                    x_thresholds.remove(0.75)

                error_dict['x']['fp'] += len(x_thresholds)
                error_dict['covar']['fp'] += len(result['covar'].keys())

            
    baseline_result_dict[fuzzy_gap] = error_dict       


100%|██████████| 7/7 [00:21<00:00,  3.04s/it]


In [45]:
pickle.dump(baseline_result_dict, open(os.path.join(RESULTS_DIR, "blend_baseline_results.dict"), "wb"))

## Subgroup discovery

In [46]:
subgroup_results_dict = {}

for fuzzy_gap in tqdm(fuzzy_gaps):
    pvals = {}
    for seed in seeds:
        with open(f"../../experiments/aaai/policy_tree_discovery/seed{seed}/blended_rdd_fixed_bw_{fuzzy_gap}.pkl", "rb") as f:
            results = pickle.load(f)    


        for idx, (result, n_tests) in enumerate(results):
            cur_trial = seed + idx
            
            x_dict = result['x']
            covar_dict = result['covar']
            x_thresholds = [np.round(c, decimals=2) for c in x_dict.keys()]
            
            pvals[cur_trial] = {
                'x_all': {},
                'x_net_benefit': {},
                'covar_all': {},
                'covar_net_benefit': {}
            }
            
            
            
            for x_cutoff in x_thresholds:
                nodes = x_dict[x_cutoff]
                if x_cutoff not in pvals[cur_trial]['x_all']:
                    pvals[cur_trial]['x_all'][x_cutoff] = []
                    pvals[cur_trial]['x_net_benefit'][x_cutoff] = []
                                  
                for node in nodes:
                    if node['llr_results'] is None:
                        print(x_cutoff)
                        continue
                    
                    pvals[cur_trial]['x_all'][x_cutoff].append(node['llr_results'].pvalues['z'])
                    
                    if node['net_benefit'] > 0:
                        pvals[cur_trial]['x_net_benefit'][x_cutoff].append(node['llr_results'].pvalues['z'])
                    
            for covar_cutoff in covar_dict.keys():
                
                if covar_cutoff not in pvals[cur_trial]['covar_all']:
                    pvals[cur_trial]['covar_all'][covar_cutoff] = []
                    pvals[cur_trial]['covar_net_benefit'][covar_cutoff] = []
                    
                nodes = covar_dict[covar_cutoff]

                for node in nodes:
                    if node['llr_results'] is None:
                        print(covar_cutoff)
                        continue
                    
                    pvals[cur_trial]['covar_all'][covar_cutoff].append(node['llr_results'].pvalues['z'])
                    
                    if node['net_benefit'] > 0:
                        pvals[cur_trial]['covar_net_benefit'][covar_cutoff].append(node['llr_results'].pvalues['z'])
        del results
        
    subgroup_results_dict[fuzzy_gap] = pvals


 86%|████████▌ | 6/7 [23:15<03:53, 233.18s/it]

0.6
0.6
0.1
0.7
0.55
0.9
0.9
0.85
0.75
0.6
0.65
0.75
0.95


100%|██████████| 7/7 [27:06<00:00, 232.33s/it]


## Compute corrected p-values

In [82]:
alpha = 0.05
low_cutoff = 0.25
upper_cutoff = 0.75

#tp_all = {}

nb_dict = {}

subgroup_trial_dict = {}

for gap in tqdm(fuzzy_gaps):
    gap_dict = {
        'upper_tp': 0,
        'lower_tp': 0,
        'fp': 0,
        'tot_tests': 0
    }
    
    trial_gap_dict = {
        'lower': [],
        'upper': []
    }
    for trial in range(500):
        trial_df = pval_df[(pval_df['trial'] == trial) & (pval_df['gap'] == gap)]
        gap_dict['tot_tests'] += trial_df.shape[0]
        
        reject, _ = pg.multicomp(trial_df['pval'].values, method='bonf', alpha=0.05)
        sig_df = trial_df[reject]
        
        if low_cutoff in list(sig_df['cutoff']):
            gap_dict['lower_tp'] += 1
            trial_gap_dict['lower'].append(1)
        else:
            trial_gap_dict['lower'].append(0)
            
        if upper_cutoff in list(sig_df['cutoff']):
            gap_dict['upper_tp'] += 1
            trial_gap_dict['upper'].append(1)
        else:
            trial_gap_dict['upper'].append(0)

            
        # remaining sig values are false positives
        fp_df = sig_df[~sig_df['cutoff'].isin([low_cutoff, upper_cutoff])]
        gap_dict['fp'] += fp_df.shape[0]
        
    nb_dict[gap] = gap_dict
    subgroup_trial_dict[gap] = trial_gap_dict
        

100%|██████████| 7/7 [00:06<00:00,  1.15it/s]


In [52]:
nb_dict

{0.2: {'upper_tp': 47, 'lower_tp': 43, 'fp': 77, 'tot_tests': 30822},
 0.3: {'upper_tp': 141, 'lower_tp': 120, 'fp': 112, 'tot_tests': 32733},
 0.4: {'upper_tp': 274, 'lower_tp': 273, 'fp': 144, 'tot_tests': 33547},
 0.5: {'upper_tp': 428, 'lower_tp': 420, 'fp': 216, 'tot_tests': 33983},
 0.6: {'upper_tp': 493, 'lower_tp': 484, 'fp': 404, 'tot_tests': 34128},
 0.7: {'upper_tp': 500, 'lower_tp': 499, 'fp': 741, 'tot_tests': 34103},
 0.8: {'upper_tp': 500, 'lower_tp': 500, 'fp': 1654, 'tot_tests': 33961}}

In [53]:
pickle.dump(nb_dict, open(os.path.join(RESULTS_DIR, "policy_tree_net_benefit.dict"), "wb"), -1)

# Load Herlands et al. results

In [77]:
HERLANDS_RESULT_PATH = "../../../herlands-lord3/results/"

herlands_dict = {}

herlands_trial_dict = {}

for gap in fuzzy_gaps:
    gap_dict = {
        'upper_tp': 0,
        'lower_tp': 0,
        'fp': 0,
        'tot_tests': 0
    }
    trial_gap_dict = {
        'lower': [],
        'upper': []
    }
    for seed in range(0, 500, 100):
        results = pickle.load(open(os.path.join(HERLANDS_RESULT_PATH, "seed{}/herlands_results_gap{}.pkl".format(seed, gap)), "rb"))
        gap_dict['lower_tp'] += sum([x[0] for x in results])
        gap_dict['upper_tp'] += sum([x[1] for x in results])
        gap_dict['tot_tests'] += sum([x[2] for x in results])

        for lower, upper, *_ in results:
            trial_gap_dict['lower'].append(int(lower))
            trial_gap_dict['upper'].append(int(upper))
        
    herlands_dict[gap] = gap_dict
    herlands_trial_dict[gap] = trial_gap_dict

In [63]:
pickle.dump(herlands_dict, open(os.path.join(RESULTS_DIR, "herlands_results.dict"), "wb"), -1)